In [1]:
import sys
import os
import warnings
os.environ['OPENBLAS_NUM_THREADS'] = '1'
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
!pip install pymorphy2
import pymorphy2
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger_ru')
#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
#for word embedding
import gensim
from gensim.models import Word2Vec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 51.5 MB/s eta 0:00:0000:0100:01


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_ru.zip.


In [3]:
import pandas as pd
import numpy as np
import time
import pyarrow as pa
import pyarrow.parquet as pq
import scipy
!pip install implicit
import implicit
import bisect
import sklearn.metrics as m
!pip install catboost
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from sklearn.calibration import calibration_curve, CalibratedClassifierCV

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

%matplotlib inline
sns.set_style('darkgrid')

In [5]:
!pip install feather-format >> none
!pip install faiss-cpu --no-cache

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 167.2 MB/s eta 0:00:00a 0:00:01


In [6]:
site_data = pd.read_csv('/data/site_information.csv')
site_data

,position,url,title,description,keywords,language
0,1024,hammerjs.github.io,hammer js hammer js,add multi touch gestures to your webpage,NaN,english
1,800,crazyimg.com,crazyimg com best free photo hosting without r...,NaN,NaN,english
2,704,mvestnik.ru,ddos guard,NaN,NaN,english
3,992,spravkus.com,spravkus com информация о компаниях россии укр...,адреса фирм телефоны режимы работы список услу...,NaN,russian
4,1025,meta.ua,access denied meta ua used cloudflare to restr...,NaN,NaN,english
...,...,...,...,...,...,...
199678,160,cgvcinemas.ru,NaN,NaN,NaN,unknown
199679,1593,moskva.svetofors.ru,NaN,NaN,NaN,unknown
199680,2835,burenergo.ru,NaN,NaN,NaN,unknown
199681,2719,mama-lev.ru,NaN,NaN,NaN,unknown


In [7]:
site_data = site_data.rename(columns={'url' : 'url_host'})
site_data

,position,url_host,title,description,keywords,language
0,1024,hammerjs.github.io,hammer js hammer js,add multi touch gestures to your webpage,NaN,english
1,800,crazyimg.com,crazyimg com best free photo hosting without r...,NaN,NaN,english
2,704,mvestnik.ru,ddos guard,NaN,NaN,english
3,992,spravkus.com,spravkus com информация о компаниях россии укр...,адреса фирм телефоны режимы работы список услу...,NaN,russian
4,1025,meta.ua,access denied meta ua used cloudflare to restr...,NaN,NaN,english
...,...,...,...,...,...,...
199678,160,cgvcinemas.ru,NaN,NaN,NaN,unknown
199679,1593,moskva.svetofors.ru,NaN,NaN,NaN,unknown
199680,2835,burenergo.ru,NaN,NaN,NaN,unknown
199681,2719,mama-lev.ru,NaN,NaN,NaN,unknown


In [8]:
site_data[['title', 'description', 'keywords']] = site_data[['title', 'description', 'keywords']].fillna('')
site_data

,position,url_host,title,description,keywords,language
0,1024,hammerjs.github.io,hammer js hammer js,add multi touch gestures to your webpage,,english
1,800,crazyimg.com,crazyimg com best free photo hosting without r...,,,english
2,704,mvestnik.ru,ddos guard,,,english
3,992,spravkus.com,spravkus com информация о компаниях россии укр...,адреса фирм телефоны режимы работы список услу...,,russian
4,1025,meta.ua,access denied meta ua used cloudflare to restr...,,,english
...,...,...,...,...,...,...
199678,160,cgvcinemas.ru,,,,unknown
199679,1593,moskva.svetofors.ru,,,,unknown
199680,2835,burenergo.ru,,,,unknown
199681,2719,mama-lev.ru,,,,unknown


In [9]:
site_data = site_data.dropna()
site_data

,position,url_host,title,description,keywords,language
0,1024,hammerjs.github.io,hammer js hammer js,add multi touch gestures to your webpage,,english
1,800,crazyimg.com,crazyimg com best free photo hosting without r...,,,english
2,704,mvestnik.ru,ddos guard,,,english
3,992,spravkus.com,spravkus com информация о компаниях россии укр...,адреса фирм телефоны режимы работы список услу...,,russian
4,1025,meta.ua,access denied meta ua used cloudflare to restr...,,,english
...,...,...,...,...,...,...
199678,160,cgvcinemas.ru,,,,unknown
199679,1593,moskva.svetofors.ru,,,,unknown
199680,2835,burenergo.ru,,,,unknown
199681,2719,mama-lev.ru,,,,unknown


In [10]:
site_data = site_data[site_data['language'] != 'unknown']
site_data

,position,url_host,title,description,keywords,language
0,1024,hammerjs.github.io,hammer js hammer js,add multi touch gestures to your webpage,,english
1,800,crazyimg.com,crazyimg com best free photo hosting without r...,,,english
2,704,mvestnik.ru,ddos guard,,,english
3,992,spravkus.com,spravkus com информация о компаниях россии укр...,адреса фирм телефоны режимы работы список услу...,,russian
4,1025,meta.ua,access denied meta ua used cloudflare to restr...,,,english
...,...,...,...,...,...,...
168542,770,critical-errorr.livejournal.com,critical errorr livejournal,с удовольствием заблуждаюсь в своих ошибочных ...,,russian
168543,3818,kodyoshibokk.ru,блог об ошибках и как их исправить,информационный блог об ошибках машин windows и...,,russian
168544,2547,journ.msu.ru,ошибка в адресе факультет журналистики мгу,,,russian
168545,3963,mistakes.ru,ошибки ру сайт об ошибках и о том как их испра...,,,russian


In [11]:
# STOPWORD REMOVAL
def stopword(string):
    a= [i for i in string.split() if i not in (stopwords.words('english') + stopwords.words('russian'))]
    return ' '.join(a)
#LEMMATIZATION
# Initialize the lemmatizer
wl = WordNetLemmatizer()
morph = pymorphy2.MorphAnalyzer()
 
# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
# Tokenize the sentence
def lemmatizer(string):    
    word_pos_tags = nltk.pos_tag(word_tokenize(string), lang='rus') # Get position tags
    list_of_words = []
    for tag in word_pos_tags:
        if tag[1] != 'NONLEX':
            list_of_words.append(morph.normal_forms(tag[0])[0])
        else:
            list_of_words.append(tag[0])
    word_pos_tags = nltk.pos_tag(list_of_words) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for tag in word_pos_tags] # Map the position tag and lemmatize the word/token
    return " ".join(a)

In [12]:
%%time
def finalpreprocess(string):
    return lemmatizer(stopword(string))
site_data[['title', 'description', 'keywords']] = site_data[['title', 'description', 'keywords']].applymap(lambda x: finalpreprocess(x))
site_data.head()

CPU times: user 59min 41s, sys: 4min 21s, total: 1h 4min 3s
Wall time: 1h 4min 17s


,position,url_host,title,description,keywords,language
0,1024,hammerjs.github.io,hammer j hammer j,add multi touch gesture webpage,,english
1,800,crazyimg.com,crazyimg com best free photo host without regi...,,,english
2,704,mvestnik.ru,ddos guard,,,english
3,992,spravkus.com,spravkus com информация компания россия украин...,адрес фирма телефон режим работа список услуга...,,russian
4,1025,meta.ua,access deny meta ua use cloudflare restrict ac...,,,english


In [13]:
site_data['info'] = site_data['title'] + ' ' + site_data['description'] + ' ' + site_data['keywords']
site_data['info']

0         hammer j hammer j add multi touch gesture webp...
1         crazyimg com best free photo host without regi...
2                                              ddos guard  
3         spravkus com информация компания россия украин...
4         access deny meta ua use cloudflare restrict ac...
                                ...                        
168542    critical errorr livejournal удовольствие заблу...
168543    блог ошибка исправить информационный блог ошиб...
168544            ошибка адрес факультет журналистика мгу  
168545                    ошибка ру сайт ошибка исправить  
168546    code radiosit ru портал код ошибка наш сайт бы...
Name: info, Length: 168547, dtype: object

In [14]:
site_data['title_description'] = site_data['title'] + ' ' + site_data['description']
site_data['title_description']

0         hammer j hammer j add multi touch gesture webpage
1         crazyimg com best free photo host without regi...
2                                               ddos guard 
3         spravkus com информация компания россия украин...
4         access deny meta ua use cloudflare restrict ac...
                                ...                        
168542    critical errorr livejournal удовольствие заблу...
168543    блог ошибка исправить информационный блог ошиб...
168544             ошибка адрес факультет журналистика мгу 
168545                     ошибка ру сайт ошибка исправить 
168546    code radiosit ru портал код ошибка наш сайт бы...
Name: title_description, Length: 168547, dtype: object

In [15]:
site_data['title_keywords'] = site_data['title']+ ' ' + site_data['keywords']
site_data['title_keywords']

0                                        hammer j hammer j 
1         crazyimg com best free photo host without regi...
2                                               ddos guard 
3         spravkus com информация компания россия украин...
4         access deny meta ua use cloudflare restrict ac...
                                ...                        
168542                         critical errorr livejournal 
168543                               блог ошибка исправить 
168544             ошибка адрес факультет журналистика мгу 
168545                     ошибка ру сайт ошибка исправить 
168546                  code radiosit ru портал код ошибка 
Name: title_keywords, Length: 168547, dtype: object

In [16]:
site_data['description_keywords'] = site_data['description'] + ' ' + site_data['keywords']
site_data['description_keywords']

0                          add multi touch gesture webpage 
1                                                          
2                                                          
3         адрес фирма телефон режим работа список услуга...
4                                                          
                                ...                        
168542    удовольствие заблуждаться свой ошибочный сужде...
168543    информационный блог ошибка машина windows проч...
168544                                                     
168545                                                     
168546    наш сайт быть публиковать самый популярный час...
Name: description_keywords, Length: 168547, dtype: object

In [17]:
site_data.to_csv('data/site_data.csv', index=False)